In [78]:
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np

df = pd.read_csv('1654342Pet shop.csv')
df.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107
4,5,170240,food,off_brand,4.6,1,NaN,40.7,2384


In [58]:
product_category_dummies = pd.get_dummies(df['product_category'])
product_quality_dummies = pd.get_dummies(df['quality'])
df_dummies = pd.concat([df,product_category_dummies,product_quality_dummies],axis=1)
df_dummies.head()
# I choose the products_sold as the reference category

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868,False,False,False,True,True,False
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625,False,False,False,True,True,False
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974,True,False,False,False,True,False
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107,False,False,False,True,True,False
4,5,170240,food,off_brand,4.6,1,NaN,40.7,2384,True,False,False,False,True,False


In [59]:
# Calculate the number of categories in 'product_category'
category_counts = df['product_category'].value_counts()

category_counts

# I'm using Health as my reference category because it has the most entries in the dataset. My first assumption will be that it sells the best aswell and that makes the regression model 
# the most interpretable with the numbers

product_category
health    851
toys      832
other     290
food      277
Name: count, dtype: int64

In [60]:
model1 = sm.ols('products_sold ~ health + food + other + toys + satisfaction + discount + retail_price + perc_physical + market_size + off_brand + premium', data = df_dummies).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:          products_sold   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     720.0
Date:                Tue, 02 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:27:36   Log-Likelihood:                -23395.
No. Observations:                2035   AIC:                         4.681e+04
Df Residuals:                    2024   BIC:                         4.687e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -8.997e+04   4311.71

In [61]:
# Check for missing values in the dataset
missing_values = df_dummies.isnull().sum()
missing_values

Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount              0
retail_price        154
perc_physical        69
market_size           0
food                  0
health                0
other                 0
toys                  0
off_brand             0
premium               0
dtype: int64

In [62]:
# Dropping the already existing missing values from the dataset. Afterwards we perform CooksD.
df_dummies_nomissingvalues = df_dummies.dropna()


In [63]:
# Before we do CooksD, we will generate another regression model to perform it on without the missing values of the original dataset
model2 = sm.ols('products_sold ~ health + food + other + toys + satisfaction + discount + retail_price + perc_physical + market_size + off_brand + premium', data = df_dummies_nomissingvalues).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:          products_sold   R-squared:                       0.782
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     767.4
Date:                Tue, 02 Apr 2024   Prob (F-statistic):               0.00
Time:                        10:27:36   Log-Likelihood:                -22268.
No. Observations:                1937   AIC:                         4.456e+04
Df Residuals:                    1927   BIC:                         4.461e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -6.197e+04   2844.69

In [64]:
# CooksD
CooksD = model2.get_influence().cooks_distance
n = len(df_dummies_nomissingvalues)
df_dummies_nomissingvalues['outlier'] = CooksD[0] > 4/n
df_dummies_nomissingvalues.head()

/tmp/ipykernel_5435/4049309850.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dummies_nomissingvalues['outlier'] = CooksD[0] > 4/n


,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,outlier
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868,False,False,False,True,True,False,False
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625,False,False,False,True,True,False,False
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974,True,False,False,False,True,False,False
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107,False,False,False,True,True,False,False
5,6,219676,other,premium,4.6,7,25.0,44.7,3735,False,False,True,False,False,True,False


In [65]:
df_dummies_nomissingvalues_true = df_dummies_nomissingvalues[df_dummies_nomissingvalues.outlier == True]
df_dummies_nomissingvalues_false = df_dummies_nomissingvalues[df_dummies_nomissingvalues.outlier == False]

In [66]:
df_dummies_nomissingvalues_true.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,outlier
196,197,83364,health,premium,3.0,4,13.0,42.9,2895,False,True,False,False,False,True,True
237,238,221811,other,off_brand,3.0,16,38.0,26.8,3773,False,False,True,False,True,False,True
384,385,209891,health,off_brand,4.7,214,15.0,53.1,2840,False,True,False,False,True,False,True
387,388,224862,toys,premium,4.8,17,29.0,38.5,2100,False,False,False,True,False,True,True
418,419,57535,other,premium,1.9,10,12.0,5.0,815,False,False,True,False,False,True,True


In [67]:
print(df_dummies_nomissingvalues[df_dummies_nomissingvalues.outlier == True])

      Unnamed: 0  products_sold product_category    quality  satisfaction  \
196          197          83364           health    premium           3.0   
237          238         221811            other  off_brand           3.0   
384          385         209891           health  off_brand           4.7   
387          388         224862             toys    premium           4.8   
418          419          57535            other    premium           1.9   
419          420         192820             toys    premium           4.9   
446          447         216386            other  off_brand           4.7   
487          488         190591             toys    premium           3.1   
493          494         190227             food    premium           4.6   
545          546         178648             toys  off_brand           4.0   
560          561          76849             toys    premium           4.7   
697          698         138834            other  off_brand           2.6   

In [68]:
# I analyzed the outliers. There were 60 of them, most of them could not be labeled as impossible. But there were some rows in which the discount percentage or the perc_physical percentage exceeded 100.
# This is impossible, thus, we will turn every value of those columns above 100 to a missing value before progressing with the assignment.
# Furthermore, the assignment tells us explicitly to return to the original dataset. So we will be going back at it again with df_dummies.

df_dummies.loc[df_dummies['discount']>100,'discount'] = np.nan
df_dummies.loc[df_dummies['perc_physical']>100,'perc_physical'] = np.nan

In [69]:
# Extra missing values have been added
df_dummies.isnull().sum()

Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount             22
retail_price        154
perc_physical        89
market_size           0
food                  0
health                0
other                 0
toys                  0
off_brand             0
premium               0
dtype: int64

In [74]:
# To deal with the categorical missing data, we add a dummy missing variable to use in regression models.

df2 = pd.get_dummies(df_dummies, dummy_na=True)

# There were no missing values for product_category. Just to make the dataset as clean as possible we will remove some of the columns which were created through the code above. We only need to keep quality_nan, the rest is redundant.

columns_to_drop = ['product_category_food', 'product_category_health', 'product_category_other', 'product_category_toys', 'quality_off_brand', 'quality_premium', 'product_category_nan']
df2 = df2.drop(columns=columns_to_drop)

df2.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,quality_nan
0,1,82640,3.6,11.0,30.0,76.4,868,False,False,False,True,True,False,False
1,2,194000,4.0,11.0,15.0,57.1,2625,False,False,False,True,True,False,False
2,3,148423,4.5,5.0,28.0,59.8,974,True,False,False,False,True,False,False
3,4,201070,4.7,2.0,15.0,33.6,3107,False,False,False,True,True,False,False
4,5,170240,4.6,1.0,NaN,40.7,2384,True,False,False,False,True,False,False


In [75]:
# For the continouos missing data we impute the data using IterativeImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputed_data = IterativeImputer().fit_transform(df2)
imputed_data = pd.DataFrame(imputed_data, columns=df2.columns)

In [79]:
imputed_data.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,quality_nan
0,1.0,82640.0,3.6,11.0,30.000000,76.4,868.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,2.0,194000.0,4.0,11.0,15.000000,57.1,2625.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,3.0,148423.0,4.5,5.0,28.000000,59.8,974.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4.0,201070.0,4.7,2.0,15.000000,33.6,3107.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,5.0,170240.0,4.6,1.0,21.037572,40.7,2384.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [83]:
# We will now check for multicollinearity using scipy.stats

import scipy.stats

import pandas as pd
from scipy.stats import pearsonr

# Select only the numerical columns (excluding dummy variables)
numerical_columns = imputed_data.select_dtypes(include=['int64', 'float64']).columns

# Initialize a DataFrame to hold the Pearson correlation results
pearson_correlations = pd.DataFrame(index=numerical_columns, columns=numerical_columns)

# Calculate Pearson correlation coefficient for each pair of numerical variables
for col1 in numerical_columns:
    for col2 in numerical_columns:
        if col1 != col2:
            # Calculate Pearson's r
            r, _ = pearsonr(imputed_data[col1], imputed_data[col2])
            pearson_correlations.at[col1, col2] = r
        else:
            # A variable's correlation with itself is always 1
            pearson_correlations.at[col1, col2] = 1.0

# Output the correlation matrix
print(pearson_correlations)



              Unnamed: 0 products_sold satisfaction  discount retail_price  \
Unnamed: 0           1.0     -0.029256    -0.034069 -0.015218     0.036127   
products_sold  -0.029256           1.0     0.471275  0.198627    -0.065139   
satisfaction   -0.034069      0.471275          1.0  0.019368    -0.009377   
discount       -0.015218      0.198627     0.019368       1.0     0.004856   
retail_price    0.036127     -0.065139    -0.009377  0.004856          1.0   
perc_physical  -0.031161      0.030781     0.022256  0.006486     0.028967   
market_size    -0.031879      0.708545    -0.005097 -0.020249     0.028238   
food            0.028231      0.146997     0.007573 -0.012185     0.024376   
health         -0.000994      0.036858     0.033842   0.02007     0.006074   
other           0.002487     -0.026013     -0.02926 -0.007453    -0.031185   
toys           -0.019942     -0.119019    -0.018842 -0.006695    -0.001046   
off_brand      -0.023713      0.100393     0.020285  0.007639   

In [90]:
# Checking individual PearsonR results to check the validity of the results from the model above.
SD = scipy.stats.pearsonr(imputed_data['satisfaction'],imputed_data['discount'])
RP_MS = scipy.stats.pearsonr(imputed_data['retail_price'],imputed_data['market_size'])
PP_RP = scipy.stats.pearsonr(imputed_data['perc_physical'],imputed_data['retail_price'])
print(SD)
print(RP_MS)
print(PP_OB)

PearsonRResult(statistic=0.01936847111565843, pvalue=0.35846033385557996)
PearsonRResult(statistic=0.0282376607108648, pvalue=0.18058613119564518)
PearsonRResult(statistic=0.028967143451790068, pvalue=0.1695792756788346)


In [ ]:
# The three values to test if the multicollinearity model works perfectly correspond with the values the model gave so we can safely assume the model is correct. Thus there are no multicollinearity issues above 80%

